<a href="https://colab.research.google.com/github/trungdung211/KalapaMedicalQA/blob/dataset/ViMedicalMCQA/%5Bcolab%5D-MedMCQA-Classification-Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Multiclass Text Classification**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/MedicalMCQA-dh22

/content/drive/MyDrive/MedicalMCQA-dh22


###**1. Load Data**

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

dataset = load_dataset('json', data_files='./data/qa_data_clean.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=True, seed=24)

 - Single Choice: A => 0, B => 1, C => 2, D => 3, E => 4
 - Multiple Choice: A => 0, B => 1, C => 2, D => 3, E => 4, AB => 5, BC => 6,..., ABC => 10,..., ABCDE => 12.

###**2. Dataloader**

In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch

letters = ["A", "B"]
# n_choices = len(letters)

# id2label = {
# }
# for i in range(1, 2**n_choices):
#     label = bin(i).replace("0b", "").rjust(5, '0')
#     id2label[i] = label

# label2id = {k: v for v, k in id2label.items()}
id2label = {
    0: "No",
    1: "A",
    2: "B",
    3: "AB"
}

label2id = {
    "No": 0,
    "A": 1,
    "B": 2,
    "AB": 3
}

num_labels = len(id2label)

def preprocess_function(examples, max_seq_length, tokenizer):
    # Tokenize the texts
    sentences = []
    labels = []
    letters = ["A", "B"]
    for example in zip(examples["question"], examples["exp"],
                       examples['opa'], examples['opb'], examples['opc'], examples['opd'], examples['ope'],
                       examples['cop']):
        question = example[0]
        context = example[1]
        ops = example[2:6]
        cop = example[7]

        for i in range(len(ops))[::2]:
            if i+1<len(ops):
                a, b = i, i+1
            else:
                a, b = i, i-1
            answer = f"A. {ops[a]}. B. {ops[b]}."

            answer_id = 3 if a in cop and b in cop else 2 if b in cop else 1 if a in cop else 0

            prompt = f"Context: {context}. Question: {question}. Answer: {answer}"
            sentences.append(prompt)
            labels.append(answer_id)

    model_inputs = tokenizer(
        sentences,
        padding="max_length",
        max_length=max_seq_length,
        truncation=True
    )
    model_inputs["labels"] = torch.tensor(labels)
    return model_inputs

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    use_fast=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    # f"./model/{model_name}/checkpoint-1000",
    num_labels=num_labels
)

# peft_config = LoraConfig(
#         r=8,
#         lora_alpha=32,
#         lora_dropout=0.05,
#         target_modules=["query", "vavlue"],
#         bias="none"
#     )

# model = get_peft_model(model, peft_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from functools import partial

processed_dataset = dataset.map(
    partial(
        preprocess_function,
        max_seq_length=256,
        tokenizer=tokenizer
    ),
    batched=True,
    load_from_cache_file=False,
    remove_columns=['question', 'exp', 'cop', 'opa', 'opb', 'opc', 'opd', 'ope'],
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/10396 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [ ]:
processed_dataset['train']['labels'][0]
# processed_dataset['train']['input_ids'][0]

2

###**3. Metric**

In [ ]:
import numpy as np
import evaluate
from transformers import EvalPrediction

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred: EvalPrediction):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = metric.compute(predictions=predictions, references=labels)
    return result

compute_metrics = compute_metrics

###**4. Trainer**

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    f"./model/{model_name}",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    save_steps = 500,
    eval_steps=500,
    save_total_limit = 3,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=processed_dataset["train"],
        eval_dataset=processed_dataset["test"],
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

In [ ]:
trainer.train(True)

Step,Training Loss,Validation Loss,Accuracy
2000,0.876800,1.009042,0.634615
3000,0.778100,1.049510,0.634423
4000,0.723400,1.032128,0.647500
5000,0.661900,1.036173,0.638462
6000,0.626900,1.077495,0.642115


KeyboardInterrupt: ignored

In [4]:
%cd /content/drive/MyDrive/MedicalMCQA-dh22

/content/drive/MyDrive/MedicalMCQA-dh22


In [5]:
import pandas as pd
df = pd.read_csv("./data/public_test_with_context.csv")

In [6]:
df.head()

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,context
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN,Phương pháp chẩn đoán rau tiền đạo Thông thườn...
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN,Nguyên nhân gây bệnh quai bị Quai bị có mặt tr...
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN,Bị đau tinh hoàn bên phải nguy hiểm như thế nà...


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

# model_name = "vinai/phobert-base"
model_name = "./model/vinai/phobert-base/checkpoint-4000"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    use_fast=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    # f"./model/{model_name}/checkpoint-1000",
    num_labels=num_labels
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
import tqdm

# convert_to_binary = {
#     0: '10000',
#     1: '01000',
#     2: '00100',
#     3: '00010',
#     4: '00001'
# }

id2Result = {
    0: [0, 0],
    1: [1, 0],
    2: [0, 1],
    3: [1, 1]
}

def remove_option_letter(choice):
    if choice[:2] in ["A.", "B.", "C.", "D.", "E.", "F."]:
        return choice[2:].strip()
    return choice

def predict_per_sample(df, model, tokenizer):
    answers = []
    # for idx, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    for idx, row in df.iterrows():
        context = row["context"].split("///")[0]
        question = row["question"]
        choices = [
            remove_option_letter(op) for op in row[["option_1", "option_2", "option_3", "option_4", "option_5", "option_6"]].tolist()
            if isinstance(op, str)
        ]
        print(choices)

        answer_list = []
        for i in range(len(choices))[::2]:
            if i+1<len(choices):
                a, b = i, i+1
            else:
                a, b = i, i-1
            answer = f"A. {choices[a]}. B. {choices[b]}."
            prompt = f"Context: {context}. Question: {question}. Answer: {answer}"

            # model_inputs = tokenizer(prompt, return_tensors="pt")
            model_inputs = tokenizer(
                prompt,
                padding="max_length",
                max_length=256,
                truncation=True,
                return_tensors="pt"
            )
            outputs = model(**model_inputs)
            prediction = torch.argmax(outputs[0], axis=1)
            results = id2Result[prediction.item()]
            print(results)
            answer_list.append(str(results[0]))
            if b > a:
                answer_list.append(str(results[1]))
        answers.append("".join(answer_list))
        # print(answer_list)


        # text_choices = " ".join(choices)
        # prompt = f"Context: {context}. Question: {question}. Choice the correct answers from: {text_choices}"
        # model_inputs = tokenizer(prompt, return_tensors="pt")
        # outputs = model(**model_inputs)
        # prediction = torch.argmax(outputs[0], axis=1)
        # answer = convert_to_binary[prediction.item()]
        # answer = answer[:len(choices)]
        # answers.append(answer)
    return answers

In [16]:
model = model.to("cpu")

In [44]:
answers = predict_per_sample(df, model, tokenizer)
answers

['Tuần 10', 'Tuần 20', 'Tuần 30', 'Tuần 40']
[0, 0]
[0, 0]
['5 tuần', '15 tuần', '25 tuần', '35 tuần']
[0, 0]
[0, 0]
['2', '3', '4', '5']
[0, 0]
[0, 0]
['Có', 'Không']
[1, 0]
['Nhiễm trùng đường tiết niệu', 'Ung thư tinh hoàn', 'Chấn thương', 'Giãn tĩnh mạch thừng tinh']
[1, 1]
[1, 1]
['Bệnh lý tiêu hóa', 'Bệnh lý hô hấp', 'Bệnh lý tim mạch', 'Bệnh lý thần kinh']
[1, 0]
[0, 0]
['Tăng cường hấp thu dinh dưỡng', 'Mất cân bằng hệ vi sinh đường ruột', 'Giảm nhu động ruột', 'Không có đáp án nào đúng']
[0, 0]
[0, 0]
['Có', 'Không']
[1, 1]
['Khám lâm sàng', 'Siêu âm qua ngả âm đạo và siêu âm ổ bụng', 'Chẩn đoán hình ảnh, thăm dò chức năng và y học hạt nhân']
[0, 0]
[0, 0]
['Có', 'Không']
[1, 1]
['Chụp X-quang ngực', 'Chụp CT', 'Thông tim', 'Chụp mạch vành', 'Điện tâm đồ (ECG)']
[1, 1]
[1, 1]
[1, 1]
['Có', 'Không', 'Không rõ']
[1, 1]
[1, 1]
['Ngủ đủ 4 tiếng/ngày', 'Ăn nhiều đồ ngọt', 'Uống đủ nước mỗi ngày 1,5 – 2 lít']
[1, 1]
[1, 1]
['Tiểu nhiều lần trong ngày', 'Nước tiểu có lẫn mủ hoặc máu

['0000',
 '0000',
 '0000',
 '10',
 '1111',
 '1000',
 '0000',
 '11',
 '000',
 '11',
 '11111',
 '111',
 '111',
 '000',
 '111',
 '111',
 '1111',
 '1111',
 '111',
 '111',
 '111',
 '111',
 '111',
 '000',
 '1111',
 '11',
 '000',
 '111',
 '1111',
 '0000',
 '1111',
 '1111',
 '111',
 '111',
 '111',
 '000',
 '1111',
 '11',
 '101',
 '11',
 '111',
 '111',
 '111',
 '1111',
 '11111',
 '111',
 '111',
 '11',
 '000',
 '111',
 '11',
 '111',
 '111',
 '111',
 '111',
 '1111',
 '11',
 '111',
 '111',
 '000',
 '111',
 '1111',
 '111',
 '0011',
 '111',
 '111',
 '111',
 '111',
 '1111',
 '0000',
 '111',
 '111',
 '111',
 '1100',
 '000',
 '000',
 '1111',
 '1111',
 '1111',
 '111',
 '11111',
 '111',
 '1111',
 '1111',
 '000',
 '00000',
 '111',
 '1111',
 '000',
 '111',
 '0000',
 '1111',
 '1111',
 '1100',
 '1111',
 '1111',
 '1111',
 '1111',
 '1111',
 '111']

In [45]:
df['answer'] = answers

In [46]:
result_df = df[["id", "answer"]]

In [47]:
result_df

,id,answer
0,level3_1,0000
1,level3_2,0000
2,level3_5,0000
3,level3_13,10
4,level3_14,1111
...,...,...
95,level4_4,1111
96,level4_9,1111
97,level4_27,1111
98,level4_28,1111


In [48]:
result_df.to_csv('./data/result_phobert_classification.csv', index=False)

#Multi-label Text Classification